In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os

from sklearn.preprocessing import LabelEncoder
from functions import equal, find_nearest, impute_immediate_mean, impute_mean_day, daylight_savings_shift, clean_weather

## Weather Data

In [109]:
cities = ['madrid', 'seville', 'barcelona', 'bilbao', 'valencia']
dfs = []
for city in cities:
    
    # Read in city dataframe
    city_df = pd.read_csv(f'../data/weather/{city}.csv', index_col=0)

    # Clean the madrid data
    city_df = clean_weather(city_df)

    # Rename columns
    city_df.columns = city_df.columns + f'_{city}'
    
    dfs.append(city_df)
    
# Create daterange
index = pd.date_range(start=dt.datetime(2015,1,1),
                      end = dt.datetime(2021,12,31, 23),
                      freq='H')
range_ = pd.DataFrame(index=index)

# Join to date range in weather data
weather = range_.join([dfs[0], dfs[1], dfs[2], dfs[3], dfs[4]])

# Drop unnecessary columns
weather.drop(columns=['index_madrid', 'index_seville', 'index_barcelona',
                      'index_bilbao', 'index_valencia'],
             inplace=True)

# Get list of categoricals and continuous variables
categorical = weather.select_dtypes(include='object').columns
continuous = weather.select_dtypes(exclude='object').columns

# Impute the mean day for continuous variables with 12 or more missing
for col in continuous:
    impute_mean_day(weather, col, 12)

# Interpolate remaining continuous Nans
weather.loc[:, continuous] = weather.loc[:, continuous].interpolate(limit=12)

# Back fill categorical nans
weather.loc[:,categorical] = weather[categorical].fillna(value='unknown')

# Back fill remaining nans (first five rows of dataset)
weather = weather.fillna(method='bfill')

# Drop duplicates in the index
weather.reset_index(inplace=True)
weather.drop_duplicates(subset='index', inplace=True)
weather.set_index('index', inplace=True)

# Drop precips cols
weather.drop(columns = weather.filter(regex='precips').columns, inplace=True)

# Lag the columns one day
weather_lag = weather.shift(1, freq='D')

# Rename weather_lag columns
lag_names = [col+'_lag' for col in weather_lag.columns]
lag_dict = dict(zip(weather_lag.columns, lag_names))
weather_lag.rename(columns=lag_dict, inplace=True)

# Export to clean folder
weather.to_csv('../data/clean/weather_clean.csv')
weather_lag.to_csv('../data/clean/weather_lag_clean.csv')

## Cross Border Transmission Data

In [119]:
# Read in France Border Data
france = pd.DataFrame()
portugal = pd.DataFrame()

for file in os.listdir('../data/cross_border/france'):
    data = pd.read_csv(f'../data/cross_border/france/{file}')
    data.drop_duplicates(subset='Time (CET)', inplace=True)
    france = pd.concat([france, data],axis=0)

# Convert Time to datetime
france['time'] = france['Time (CET)'].apply(lambda x: x[:13])
france['time'] = pd.to_datetime(france['time'], format="%d.%m.%Y %H")
france = france.set_index('time').drop(columns='Time (CET)')

# Read in Portugal Border Data
for file in os.listdir('../data/cross_border/portugal'):
    data = pd.read_csv(f'../data/cross_border/portugal/{file}')
    data.drop_duplicates(subset='Time (CET)', inplace=True)
    portugal = pd.concat([portugal, data],axis=0)
portugal['time'] = portugal['Time (CET)'].apply(lambda x: x[:13])
portugal['time'] = pd.to_datetime(portugal['time'], format="%d.%m.%Y %H")
portugal = portugal.set_index('time').drop(columns='Time (CET)')

# Join France and Portugal Data
border = portugal.join(france)
border.fillna(method='ffill', inplace=True)
cols = dict(zip(border.columns,['transmission_ps', 
                                'transmission_sp', 
                                'transmission_fs',
                                'transmission_sf']))

border.rename(columns=cols, inplace=True)

# lag border data and rename cols
border_lag = border.shift(1, freq='D')
lag_names = [col+'_lag' for col in border_lag.columns]
lag_dict = dict(zip(border_lag.columns, lag_names))
border_lag.rename(columns=lag_dict, inplace=True)

# Export to clean folder
border.to_csv('../data/clean/border_clean.csv')
border_lag.to_csv('../data/clean/border_lag_clean.csv')

## Generation

In [133]:
gen = pd.DataFrame()
for file in os.listdir('../data/generation'):
    load = pd.read_csv(f'../data/generation/{file}')
    load.drop_duplicates(subset='MTU', inplace=True)
    gen = pd.concat([gen, load], axis=0)
    
# Get rid of columns that do not contain any information
gen = gen.drop(columns = gen.loc[:,gen.nunique()<=1].columns)

# Convert Time to datetime
gen['time'] = pd.to_datetime(gen[gen.columns[0]].apply(lambda x: x[:13]),
                             format="%d.%m.%Y %H")

# Set index to the time col
gen.set_index('time', inplace=True)

# Drop MTU col
gen.drop(columns='MTU', inplace=True)

# Rename cols
gen.columns = gen.columns.map(lambda x: ('generation '+ x[:-26]).lower())
gen.rename(columns={'generation hydro pumped storage ': 'generation hydro pumped storage consumption'},
           inplace=True)

# Impute Immediate Mean for NaNs
for col in gen.columns:
    indices = gen.loc[gen[col].isna()].index
    for i in indices:
        gen.loc[i,col] = impute_immediate_mean(gen[col], i)

# Rename cols
col_map = dict(zip(gen.columns, [col.split(' ')[-1] for col in gen.columns]))
gen.rename(columns=col_map, inplace=True)

# Shift columns
gen_lag = gen.shift(1, freq='D')

# Rename Lag Columns
col_map = dict(zip(gen.columns, [col + '_lag' for col in gen.columns]))
gen_lag.rename(columns=col_map, inplace=True)

# Export to clean folder
gen.to_csv('../data/clean/generation_clean.csv')
gen_lag.to_csv('../data/clean/generation_lag_clean.csv')

## Load Forecast and Actual

In [142]:
load_forecast = pd.DataFrame()
for file in os.listdir('../data/load'):
    load = pd.read_csv(f'../data/load/{file}')
    if load.columns[0] == 'Time (CET/CEST)':
        load.rename(columns={'Time (CET/CEST)':'Time (CET)'}, inplace=True)
    load.drop_duplicates(subset=load.columns[0], inplace=True)
    load_forecast = pd.concat([load_forecast, load], axis=0)

# Convert Time to datetime
load_forecast['time'] = pd.to_datetime(load_forecast['Time (CET)'].apply(lambda x: x[:13]),
                                       format="%d.%m.%Y %H")

# Set index to the time col
load_forecast.set_index('time', inplace=True)

# Drop 'Time (CET)' col
load_forecast.drop(columns='Time (CET)', inplace=True)

# Rename cols
load_forecast.rename(columns = {'Day-ahead Total Load Forecast [MW] - BZN|ES':'load_forecast',
                                'Actual Total Load [MW] - BZN|ES':'load_actual'}, 
                     inplace=True)

# Impute Immediate Mean for NaNs
for col in load_forecast.columns:
    indices = load_forecast.loc[load_forecast[col].isna()].index
    for i in indices:
        load_forecast.loc[i,col] = impute_immediate_mean(load_forecast[col], i)
        
# Shift load_actual
load_forecast_lag = load_forecast.join(load_forecast.load_actual.shift(1, 'D'), how='outer', lsuffix='_drop')
load_forecast_lag.drop(columns='load_actual_drop', inplace=True)

# Rename Lagged cols
load_forecast_lag.rename(columns={'load_actual':'load_actual_lag'}, inplace=True)

# Export to clean folder
load_forecast.to_csv('../data/clean/load_forecast_clean.csv')
load_forecast_lag.to_csv('../data/clean/load_forecast_lag.csv')

## Wind and Solar Forecast

In [148]:
ws = pd.DataFrame()
for file in os.listdir('../data/wind_solar_day_ahead'):
    load = pd.read_csv(f'../data/wind_solar_day_ahead/{file}')
    if load.columns[0] != 'Time (CET)':
        load.rename(columns={load.columns[0]:'Time (CET)'}, inplace=True)
    load.drop_duplicates(subset=['Time (CET)'], inplace=True)
    ws = pd.concat([ws, load], axis=0)

# Convert Time to datetime
ws['time'] = pd.to_datetime(ws[ws.columns[0]].apply(lambda x: x[:13]),
                                       format="%d.%m.%Y %H")

# Set index to the time col
ws.set_index('time', inplace=True)

# Drop 'Time (CET)' col
ws = ws[['Generation - Solar  [MW] Day Ahead/ BZN|ES',
         'Generation - Wind Onshore  [MW] Day Ahead/ BZN|ES']].copy()

# Rename cols
ws.rename(columns = {'Generation - Solar  [MW] Day Ahead/ BZN|ES':'solar_forecast',
                     'Generation - Wind Onshore  [MW] Day Ahead/ BZN|ES':'wind_forecast'}, 
                     inplace=True)

# 2020-05-01 wind forecast is missing, impute average for that day in may
avg_w = ws['wind_forecast'].groupby(by=[ws.index.month, 
                                        ws.index.day,
                                        ws.index.hour]).mean().loc[(5,1)]
for i, time in enumerate(ws.loc['2020-05-01'].index):
    ws.loc[time, 'wind_forecast'] = avg_w[i]
    
# Impute the immediate mean for remaining NaNs
for col in ws.columns:
    indices = ws.loc[ws[col].isna()].index
    for i in indices:
        ws.loc[i,col] = impute_immediate_mean(ws[col], i)
        
# Shift columns
ws_lag = ws.shift(1, freq='D')

# Rename Lag Columns
col_map = dict(zip(ws_lag.columns, [col + '_lag' for col in ws_lag.columns]))
ws_lag.rename(columns=col_map, inplace=True)

# Export to clean folder
ws.to_csv('../data/clean/wind_solar_clean.csv')
ws_lag.to_csv('../data/clean/wind_solar_clean_lag.csv')

## Generation Forecast Day ahead

In [162]:
# Load and combine all day ahead price data into single dataframe
gen_forecast = pd.DataFrame()
for file in os.listdir('../data/gen_forecast'):
    load = pd.read_csv(f'../data/gen_forecast/{file}')
    load.drop_duplicates(subset='MTU', inplace=True)
    gen_forecast = pd.concat([gen_forecast, load], axis=0)
    
# Convert Time to datetime
gen_forecast['time'] = gen_forecast['MTU'].apply(lambda x: x[:16])
gen_forecast['time'] = pd.to_datetime(gen_forecast['time'])

# Convert Time to datetime
gen_forecast['time'] = pd.to_datetime(gen_forecast[gen_forecast.columns[0]].apply(lambda x: x[:13]),
                                      format="%d.%m.%Y %H")

# Drop unused columns and rows
gen_forecast.drop(columns='MTU', inplace=True)

# Set index to the time col
gen_forecast.set_index('time', inplace=True)


# Rename cols
gen_forecast.rename(columns={'Scheduled Generation [MW] (D) - BZN|ES':'generation_forecast',
                             'Scheduled Consumption [MW] (D) - BZN|ES':'consumption_forecast'},
                    inplace=True)

for col in gen_forecast.columns:
    
    # Impute the mean of the nearest known date by hour
    impute_mean_day(gen_forecast, col, 24)
    
    # Fill remaining Nans
    gen_forecast[col].fillna(method='bfill', inplace=True)

# Export to clean folder
gen_forecast.to_csv('../data/clean/gen_forecast_clean.csv')

## Prices and Day Ahead Prices

In [26]:
# Change directory and get all files in data/price directory
os.chdir('../data/price')
files = os.listdir()

# Read all files in directory
prices = pd.DataFrame()
for file in files:
    prices =pd.concat([prices, pd.read_csv(file, delimiter=';', index_col=5, parse_dates=True)])

# Join on date_range to make sure we aren't missing any rows
price_df = pd.DataFrame(
    index=pd.date_range(start=dt.datetime(2015,1,1),
                        end=dt.datetime(2021,12,31,23),
                        freq='H',
                        tz='CET')
)

# Create cols dictionary to rename cols
cols = {'Hourly average price final sum of components':'price_actual',
        'Hourly average price Day Ahead market component':'price_day_ahead'}

cols = {'Hourly average price final sum of components':'price_actual',
       'Hourly average price intraday market tech. constraints component':'price_intraday_tech',
       'Hourly average price PBF technical constraints component':'price_PBF_tech',
       'Hourly average price real time technical constraints component':'price_rt_tech',
       'Hourly average price intraday market component':'price_intraday_market',
       'Hourly average price Day Ahead market component':'price_day_ahead',
       'Hourly average price Upward reserve power component':'price_upward_reserve',
       'Hourly average price secondary reserve component':'price_sec_reserve',
       'Hourly average price measured imbalances component': 'price_measured_imbalances',
       'Hourly average price imbalances net value component':'price_imbalances_net',
       'Hourly average price capacity payment component':'price_capacity_payment',
       'Hourly average price P.O.14.6 balance component':'price_P0146_balance',
       'Hourly average price Generic Units Nom. Failure component':'price_generic_failure',
       'Hourly average price interruptibility service component':'price_interupt_service',
       'Hourly average price power factor control component':'price_power_factor',
       'Hourly average price balance energy failure component':'price_balance_failure'}

# Create price dictionary to hold column data
price_dict = {cols[col]:prices.loc[prices.name==col, 'value'] for col in cols.keys()}

# For each column, add to price_df
for key in price_dict.keys():
    price_dict[key] = price_dict[key].rename(key)
    price_df = price_df.join(price_dict[key].groupby(by=price_dict[key].index).mean())

# Drop all the duplicates
price_df = price_df.groupby(by=price_df.index).mean().copy()

# Make timezone unaware, shift by an hour to offset the unaware tz
price_df.index = price_df.index.tz_convert(None)
price_df = price_df.shift(1, freq='H')

#Shift prices, drop cols, and rename
#price_df = price_df.join(price_df[['price_actual', 'price_day_ahead']].shift(-1, freq='D'), rsuffix='_tomorrow')
#price_df.drop(columns='price_day_ahead', inplace=True)
#price_df.rename(columns={'price_actual_tomorrow':'price_tomorrow',
#                         'price_day_ahead_tomorrow': 'price_forecast_tomorrow'}, inplace=True)
price_df.fillna(value=0, inplace=True)
# Change directory back to ../script
os.chdir('../../scripts')

# Export to clean folder
price_df.to_csv('../data/clean/price_df_clean.csv')

## Combine dataframes

In [209]:
# Create df with date_range and copy it to second df
df = pd.DataFrame(
    index=pd.date_range(start = dt.datetime(2014,12,31),
                        end = dt.datetime(2022,1,1,23),
                        freq = 'H')
)
df_lag = df.copy()

# Join all non-lagged data to create df, and lagged data to created lagged
df = df.join([weather, border, gen, load_forecast, ws, gen_forecast, price_df])
df_lag = df_lag.join([weather_lag, border_lag, gen_lag, load_forecast, ws_lag, gen_forecast, price_df])


# Drop last day since no price data for 2022
df.dropna(inplace=True)
df_lag.dropna(inplace=True)

# Export to clean folder
df.to_csv('../data/clean/df_clean.csv')
df_lag.to_csv('../data/clean/df__clean_lag.csv')